# io — Core tools for working with streams

>Category : Generic Operating System Services     
>Source : https://docs.python.org/3/library/io.html  
>See also :  
>- open(), os, os.path, fileinput, tempfile, shutil.  
>- sys : contains the standard IO streams: sys.stdin, sys.stdout, and sys.stderr.


The io module provides Python’s main facilities for dealing with various types of I/O. There are three main types of I/O: 

- text I/O
- binary I/O
- raw I/O 
    

These are generic categories, and various backing stores can be used for each of them. A concrete object belonging to any of these categories is called a **file object**. Other common terms are **stream and file-like object**.

Independently of its category, each concrete stream object will also have various capabilities: it can be **read-only, write-only, or read-write**. It can also allow **arbitrary random access** (seeking forwards or backwards to any location), or only **sequential access** (for example in the case of a socket or pipe).

All streams are careful about the type of data you give to them. For example giving a str object to the write() method of a binary stream will raise a TypeError. So will giving a bytes object to the write() method of a text stream.

In [1]:
import io
import os
import os.path

if not os.path.exists('test_io'):
    os.mkdir('test_io')

## Various I/O
### Text I/O
**Text I/O expects and produces str objects.** This means that whenever the backing store is natively made of bytes (such as in the case of a file), encoding and decoding of data is made transparently as well as optional translation of platform-specific newline characters.

The text stream API is described in detail in the documentation of TextIOBase.

The easiest way to create a text stream is with open(), optionally specifying an encoding:

In [2]:
with open('test_io/myfile.txt', 'r', encoding='utf-8') as f: # Actually, utf-8 is the default str format.
    print(f.read())

this is a text file.
这是一个字符串文件。


**In-memory** text streams are also available as StringIO objects:

In [3]:
f = io.StringIO("some initial text data")
print(f)

### Binary I/O

**Binary I/O (also called buffered I/O) expects and produces bytes objects**. No encoding, decoding, or newline translation is performed. This category of streams can be used for all kinds of non-text data, and also when manual control over the handling of text data is desired.

The binary stream API is described in detail in the docs of BufferedIOBase.

The easiest way to create a binary stream is with open() with 'b' in the mode string:

In [4]:
with open("test_io/favicon.ico", "rb") as f:  # favicon.ico is a non-text file.
    print(f.read(10))  # Read ten bytes.

b'\x00\x00\x01\x00\x01\x00\x10\x10\x00\x00'


In [5]:
# Read text file as bytys format, note that the English character is the origin bytes,
# and the Chinese character is encoded in utf-8.
f = open("test_io/myfile.txt", "rb")
print(f.read())
f.close()

b'this is a text file.\n\xe8\xbf\x99\xe6\x98\xaf\xe4\xb8\x80\xe4\xb8\xaa\xe5\xad\x97\xe7\xac\xa6\xe4\xb8\xb2\xe6\x96\x87\xe4\xbb\xb6\xe3\x80\x82'


In [6]:
# Use bytes format write Chinese to str file.
with open("test_io/mybyte.txt", "wb") as f:
    f.write('这是通过写字节加入的中文'.encode('utf-8'))  # This is equivalent to "b'\x89\be\..'"
with open("test_io/mybyte.txt", "r") as f:
    print(f.read())

这是通过写字节加入的中文


In-memory binary streams are also available as BytesIO objects:

In [7]:
f = io.BytesIO(b"some initial binary data: \x00\x01")
print(f)

### Raw I/O

**Raw I/O (also called unbuffered I/O)** is generally used as a low-level building-block for binary and text streams; it is rarely useful to directly manipulate a raw stream from user code. 

The raw stream API is described in detail in the docs of RawIOBase.

Nevertheless, you can create a raw stream by opening a file in binary mode with buffering disabled:

In [8]:
with open("test_io/favicon.ico", "rb", buffering=0) as f:  # favicon.ico is a non-text file.
    print(f.read(10))  # Read ten bytes.

b'\x00\x00\x01\x00\x01\x00\x10\x10\x00\x00'


## Common Usage 

Given that dealing with strings is the most common scenario, we primarily talk about the usage of TextIO. The usage of other I/O format is similar.

### open() & close()
Character|	Meaning
:----------|:----------
'r'	|open for reading (default)
'w'	|open for writing, truncating the file first
'x'	|open for exclusive creation, failing if the file already exists
'a'	|open for writing, appending to the end of the file if it exists
'b' |binary mode
't'	|text mode (default)
'+'	|open a disk file for updating (reading and writing)
'U'	|universal newlines mode (deprecated)

In [9]:
f = open("test_io/myfile.txt", "r") 
print(f)
f.close()

<_io.TextIOWrapper name='test_io/myfile.txt' mode='r' encoding='UTF-8'>


### read
Method|Description
:-----|:------
read(size)|Read and return at most size characters from the stream as a single str. If size is negative or None, reads until EOF.
readline(size=-1)|Read until newline or EOF and return a single str. If the stream is already at EOF, an empty string is returned.
readlines(hint=-1)|Read and return a list of lines from the stream. hint can be specified to control the number of lines read
readable()|Return True if the stream can be read from. If False, read() will raise OSError.

In [10]:
f = open("test_io/myfile.txt", "rt")
f.read()

'this is a text file.\n这是一个字符串文件。'

In [11]:
f.close()
f = open("test_io/myfile.txt", "rt")
f.readline()

'this is a text file.\n'

In [12]:
f.readline()

'这是一个字符串文件。'

In [13]:
f.close()
f = open("test_io/myfile.txt", "rt")
f.readlines()

['this is a text file.\n', '这是一个字符串文件。']

In [14]:
f.readable()

True

In [15]:
f.writable()

False

In [16]:
f.close()

### Write
Method|Description
:-----|:------
write(s)|Write the string s to the stream and return the number of characters written.
writelines(lines)|Write a list of lines to the stream. Line separators are not added, so it is usual for each of the lines provided to have a line separator at the end.
writable()|Return True if the stream supports writing. If False, write() and truncate() will raise OSError.

In [17]:
f = open("test_io/mywrite.txt", "w")
f.write("The first line is written by write()\n")  # Note that this won't add \n in the end.

37

In [18]:
lines = ["The second line is written by writelines([list_of_lines_one])\n",
         "The third line is written by writelines([list_of_lines_two])\n"]
f.writelines(lines)

In [19]:
f.writable()

True

In [20]:
f.readable()

False

In [21]:
f.close()

In [22]:
with open("test_io/mywrite.txt", "r") as f:
    print(f.read())

The first line is written by write()
The second line is written by writelines([list_of_lines_one])
The third line is written by writelines([list_of_lines_two])

